In [1]:
!sh get_data.sh

fatal: destination path 'ConvFinQA' already exists and is not an empty directory.


In [2]:
!unzip ConvFinQA/data.zip -d ConvFinQA/

Archive:  ConvFinQA/data.zip
replace ConvFinQA/data/test_private.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [4]:
import json
from retrieve import vector_store
from utils import format_document
def parse_convfinqa_dataset(filepath, limit: int = None):
    with open(filepath, 'r') as f:
        data = json.load(f)
    docs = []
    
    if limit:
        data = data[:limit]

    for entry in data:
        doc = format_document(entry)
        docs.append(doc)
    
    return docs

In [7]:
from retrieve import COLLECTION_NAME, chromadb_client, sentence_transformer_ef

try:
    chromadb_client.delete_collection(name=COLLECTION_NAME)
    chromadb_client.clear_system_cache()
except ValueError:
    pass

db = chromadb_client.create_collection(name=COLLECTION_NAME, embedding_function=sentence_transformer_ef)


DATA_PATH = 'ConvFinQA/data/train.json'
docs = parse_convfinqa_dataset(DATA_PATH, limit=1000)
ids = [doc.id for doc in docs]
texts = [doc.page_content for doc in docs]
metadatas = [doc.metadata for doc in docs]

db.add(ids=ids, documents=texts, metadatas=metadatas)